## Enunciado

Para este ejercicio se va a imaginar que se trabaja para una empresa de envíos de comida, presente en todo el territorio nacional, con miles de pedidos cada día. Dicha empresa tiene un fichero histórico con todas las peticiones de comida que los clientes han realizado mediante el chat de su web en los últimos meses. Necesitan __analizar en tiempo real qué comidas están pidiendo los usuarios y qué ingredientes tenían__, ya que en la cadena de stock de alimentos es necesario realizar una previsión para no quedarse sin platos cocinados. Se ha calculado que el impacto en las ventas cada vez que uno de los platos deja de estar disponible es del 7% de pérdidas en esa semana, debido al abandono de la web de pedidos por parte del cliente. Por tanto, es de vital importancia poder realizar automáticamente estimaciones al respecto.

El objetivo es programar una __función que reciba como input un texto de usuario y devuelva los fragmentos de texto (chunks) que hagan referencia a las comidas y cantidades que ha solicitado__. No es necesario, ni es el objetivo de este ejercicio, construir un clasificador de intención previo a esta función, sino simplemente una función que presuponemos recibe una frase con la intención 'Pedir_comida'. Tampoco es objetivo normalizar la salida (por ej.: no es necesario convertir 'tres' a '3' ni 'pizzas' a 'pizza'). Es, por tanto, un ejercicio de mínimos.

Por tanto, la __salida de la función__ será un __array con diccionarios de 2 elementos (comida y cantidad)__. Cuando una cantidad no sea detectada, se pondrá su valor a '1' como valor por defecto.

El alumno deberá usar un __NaiveBayesClassifier__, en lugar del MaxEntClassifier, para localizar los elementos descritos anteriormente (comida y cantidad). Si el alumno no es capaz de construir un NaiveBayesClassifier —necesario para obtener un 10 en la práctica—, puede realizarlo mediante unigram o bigram tagger —para obtener un 9— o si no mediante RegexParser —un 7—.

Se deberá comenzar la práctica por el nivel más básico de dificultad (RegexParser) y, en caso de conseguirlo, añadir los siguientes niveles de forma sucesiva. De esta forma, el entregable contendrá todas y cada una de las tres formas de solucionar el problema. No basta, por tanto, con incluir, por ejemplo, únicamente un NaiveBayesClassifier, hay que incluir también las otras dos formas si se quiere obtener la máxima puntuación. Se trata simplemente de una práctica y, por tanto, no se espera como resultado un sistema de alta precisión listo para usar en producción, sino simplemente una aproximación básica que permita ejecutar las tres formas de resolver el problema.

## Imports

In [1]:
import nltk
import re
import pprint
from nltk.chunk import *
from nltk.chunk.util import *
from nltk.chunk.regexp import *
from nltk import Tree

## 1. Se crea un corpus de frases de ejemplo.

In [2]:
corpus = ['Me pones un pincho de tortilla, por favor?',
          'quiero una pizza margherita',
          'quiero un perrito caliente con patatas',
          'voy a pedir un sandwich de atún',
          'quiero 2 manzanas y una tarta de limón',
          'Queremos tres tortillas de patata y unos macarrones con queso',
          'una sopa de verduras',
          'Un plato de pescado',
          'quiero pedir tres raciones de calamares y dos de croquetas',
          'pizza con peperoni y aceitunas, gracias',
          'Tomaré una mousse de limón',
          'Bocadillo de jamón y queso',
          'quiero pedir un sandwich de atún y 3 hamburguesas con queso',
          'Me gustaría pollo asado y tres bocadillos de tortilla, muchas gracias',
          'quiero dos raciones de calamares',
          'tallarines carbonara',
          'tortilla de calabacín',
          'Dos bizcochos de naranja',
          'dos pizzas de peperoni y una pizza de queso, por favor',
          'Quiero una pizza con anchoas',
          'quiero pedir 4 bocadillo de jamón',
          'Me pones un arroz con tomate',
          'me gustaría una paella',
          'Dos filetes empanados con patatas y un bocadillo de jamón, gracias',
          'Quiero pedir una tarta de manzana',
          'un batido de chocolate y tres tartas de  zanahoria',
          'Una ración de queso y jamón',
          'Tres galletas de coco',
          'quiero un mousse de chocolate y un batido de coco',
          'Un plato de chipirones',
          'Quiero 2 flanes con nata',
          'Dos naranjas y una manzana',
          'Me pones un bizcocho con nueces y dos galletas de chocolate',
          'me pones una tostada de aguacate y 1 tostada de salmón',
          'Voy a pedir salmón con verduras y una pizza carbonara',
          'Quiero dos platos de macarrones con tomate y un filete de ternera',
          'Me gustaría una sopa de marisco',
          'Quiero dos perritos calientes y unas patatas con queso y bacon',
          'Me pones dos filetes de lomo con patatas',
          'Quiero un puré de calabaza y un filete de pescado',
          'Unos nuggets y unas patatas fritas',
          'Quiero dos sopas de pescado y un solomillo',
          'Me pones 4 platos de pasta con tomate y 2 filetes de pescado',
          'quiero macarrones con queso',
          'Vamos a pedir tres batidos de plátano',
          'una hamburguesa con bacon y queso',
          'Dos ensaladas con aceitunas y anchoas',
          'tres pizzas con ajo y una lasaña',
          'Quiero cuatro raciones de carne estofada',
          'Me gustaría pedir dos solomillos con patatas',
          'Nachos con guacamole y queso',
          'Una tarta de chocolate y una tarta de queso',
          'Queremos tres ensaladas con tomate y jamón y dos bocadillos de lomo',
          'un helado de vainilla',
          'Un yogur de limón y una tarta de chocolate',
          'quiero guisantes con jamón',
          'dos filetes de pescado y unos espaguetis con tomate',
          'una sopa de fideos',
          'pediré 3 batidos de vainilla y 1 helado de fresa',
          'quiero pedir pasta con tomate y filetes rusos',
          'voy a pedir 2 filetes de pescado, 1 hamburguesa con bacon y queso y 2 tortillas con calabacín',
          'un huevo frito con patatas y dos bocadillos de jamón y queso, por favor',
          'dos perritos calientes y un sandwich de atún y queso',
          'arroz con tomate y filete de lomo con patatas, gracias',
          'vamos a pedir tres galletas con chocolate, 2 tartas de limón y 1 café',
          'quiero pedir 3 cafés con leche, 2 tartas de zanahoria, un flan y 2 helados de chocolate, por favor',
          'ponme 2 filetes con patatas y una cerveza',
          'tres tartas de queso',
          'quiero dos manzanas y una pera',
          'Queremos dos bocadillos de tortilla y dos bocadillos de jamón',
          'un sandwich de lomo',
          'ensalada de aguacate y filete de pollo',
          'Quería un filete de lomo',
          'Quiero dos hamburguesas con cebolla y bacon',
          'dos zumos de naranja y dos tostadas con tomate',
          'dos refrescos, tres cervezas y ensalada de aguacate',
          'Tres cafés y una galleta con chocolate',
          'queremos un bocadillo de jamón, un boadillo de queso, un bocadillo de lomo y tres bocadillos de tortilla',
          'Dos helados de chocolate',
          'Quiero una ensalada con anchoas',
          'voy a pedir puré de calabaza',
          'quiero dos tostadas con salchichón',
          'quiero lubina con verduras',
          'me pones dos ensaladas con pollo y 3 filetes de lomo',
          'voy a pedir patatas fritas',
          'quiero que me pongas un solomillo, un filete de lomo con patatas y 2 tartas de queso',
          'traeme 3 bizcochos de naranja',
          'ponme un puré de verduras, 2 arroz con tomate y filete con patatas',
          'quiero ensaladilla rusa',
          '2 raciones de jamón y queso',
          'vamos a pedir 5 cervezas y 5 hamburguesas con queso y bacon',
          'bizcocho de almendras',
          'dos ensaladas con cacahuetes y una pizza con aceitunas',
          'Tres ensaladas con queso y dos huevos fritos, gracias',
          'Tomaré una mandarina y un plátano',
          'Un batido de fresa y dos yogures de coco',
          'Cinco purés de verduras y dos filetes de ternera con patatas',
          'quiero pedir melón con jamón',
          'risotto con queso',
          'dos sandwiches de queso y cebolla, 1 bocadillo de chorizo y una hamburguesa de pollo',
          'Quiero tres mandarinas, una manzana y dos tartas de queso',
          'Patatas con queso y bacon',
          'ponme 2 hamburguesas con bacon, unas patatas fritas y unos aros de cebolla',
          'quiero pedir nachos con queso y una cerveza',
          'me pones un batido de fresa, un zumo de melocotón y 2 tartas de manzana',
          '3 raciones de queso y una ración de croquetas',
          'un solomillo con pimientos y una lubina',
          'quiero pedir pasta con tomate y una pizza carbonara',
          'queremos pan con ajo, pasta boloñesa y pizza con anchoas',
          'Pasta con aguacate y gambas',
          'quiero un risotto con champiñones',
          'quiero pedir 2 raciones de croquetas y un arroz con verduras',
          'Voy a pedir tres platos de arroz con gambas',
          'Quiero dos tostadas con tomate y aguacate',
          'Me gustaría pedir unas gambas',
          'quiero unos tallarines con gambas y un rollito de verduras, por favor',
          'Me pone tres platos de filetes con guisantes',
          'Querría dos filetes empanados con verduras',
          'quiero tres filetes de pollo y 2 helados de fresa',
          'vamos a pedir una cerveza, un refresco, un zumo de naranja y dos raciones de patatas fritas',
          'fresas con nata',
          'voy a pedir dos fajitas de ternera y unos nachos con queso',
          'tres macarrones carbonara',
          'quiero dos burritos y unos nachos',
          'Vamos a pedir una hamburguesa con queso y dos pizzas con anchoas y peperoni',
          'Nos gustaría tomar dos sopas de fideos y una ensalada césar',
          'Pediré unos aros de cebolla y una hamburguesa con bacon',
          'Quiero una sopa castellana',
          'Vamos a pedir tres filetes con puré de patata',
          'quiero dos bocadillos de salchicas y un bocadillo de tortilla, gracias',
          'una tortilla de patatas y una ensalada',
          'Dos perritos calientes y unas patatas con queso',
          'Quiero cuatro tiramisús',
          'voy a pedir dos refrescos, un bocadillo de calamares y un filete de lomo',
          'me pones un pincho de tortilla, un puré de calabacín y un pollo asado',
          'quiero que me traigas dos hamburguesas con cebolla y unas patatas fritas',
          'voy a pedir ensalada de tomate, 2 hamburguesas de pollo y tarta de chocolate',
          'quiero pedir guisantes con jamón y melón, por favor',
          '2 batidos de chocolate, 1 zumo de naranja y 2 bocadillos de queso',
          'quiero un risotto de setas y un solomillo con verduras']
 

In [3]:
len(corpus)

140

## 2. Entrenar un tagger para el español.

Cargamos todas las frases anotadas del corpus CESS

In [4]:
from nltk.corpus import cess_esp

sents = cess_esp.tagged_sents()

Creamos un conjunto de entrenamiento y otro de prueba.

Metemos en el conjunto de entrenamiento el 90% de las frases, y el restante 10% en el conjunto de test

In [5]:
training = []
test = []

for i in range(len(sents)):
    if i % 10:
        training.append(sents[i])
    else:
        test.append(sents[i])

Entrenamos un POS tagger con el corpus en español, en este caso se ha elegido un __Hidden Markov Models__ porque es el más completo y no necesita de backoffs pues basa la asignación de etiquetas a cada palabra en la probabilidad conjunta que tiene la frase completa.

In [6]:
from nltk.tag.hmm import HiddenMarkovModelTagger

hmm_tagger = HiddenMarkovModelTagger.train(training)

Evaluamos sobre el conjunto de test que no usamos para el entrenamiento, para ver qué porcentaje de acierto hemos conseguido.

In [7]:
print ('Acierto con HMMs:',hmm_tagger.evaluate(test)*100)

Acierto con HMMs: 89.88905831011094


## Probamos el tagger con las 5 primeras frases de nuestro corpus de prueba

In [8]:
for sentence in corpus[:5]:
    
    tokens = nltk.word_tokenize(sentence)
    tagged_HMM = hmm_tagger.tag(tokens)
    
    print('\033[1m Análisis con HMM de la frase: \033[0m', sentence, '\n')
    print(tagged_HMM, '\n')

 Análisis con HMM de la frase:  Me pones un pincho de tortilla, por favor? 

[('Me', 'pp1cs000'), ('pones', 'vmip3s0'), ('un', 'di0ms0'), ('pincho', 'ncms000'), ('de', 'sps00'), ('tortilla', 'ncfs000'), (',', 'Fc'), ('por', 'sps00'), ('favor', 'ncms000'), ('?', 'Fit')] 

 Análisis con HMM de la frase:  quiero una pizza margherita 

[('quiero', 'sps00'), ('una', 'di0fs0'), ('pizza', 'ncfs000'), ('margherita', 'aq0fs0')] 

 Análisis con HMM de la frase:  quiero un perrito caliente con patatas 

[('quiero', 'sps00'), ('un', 'di0ms0'), ('perrito', 'ncms000'), ('caliente', 'aq0cs0'), ('con', 'sps00'), ('patatas', 'ncfp000')] 

 Análisis con HMM de la frase:  voy a pedir un sandwich de atún 

[('voy', 'rg'), ('a', 'sps00'), ('pedir', 'vmn0000'), ('un', 'di0ms0'), ('sandwich', 'ncms000'), ('de', 'sps00'), ('atún', 'da0fs0')] 

 Análisis con HMM de la frase:  quiero 2 manzanas y una tarta de limón 

[('quiero', 'da0mp0'), ('2', 'Z'), ('manzanas', 'ncmp000'), ('y', 'cc'), ('una', 'di0fs0'), ('t

## 3. Construir un Regex Parser que detecte comidas y cantidades

Creamos las expresiones regulares necesarias para reconocer cúando hablamos de comidas y cantidades. La estrucutura que queremos identificar es la siguiente:
- Numeral cardinal (cuatro, cinco) o determinante indefinido o numeral (un, una) o un número entero (2, 3) antecediendo al alimento que nos va a indicar la __cantidad__, ej '__una__ tortilla... ' (en ocasiones no aparece, entonces supondremos que es 1).
- Sustantivo común que define el __alimento__, ej '__tortilla__...'.
- Sustantivo común, det. adj. o adj. calificativo que acompaña al alimento y se refiere al __ingrediente__, ej 'tortilla de __patata__...'.

Las etiquetas que estamos buscando son:
- Las etiquetas que pertenecen a __determinantes indefinidos__ son de la forma <code>'di___'</code>.
- Las que pertenecen a __determinantes numerales__ son de la forma <code>'dn___'</code>
- En el caso de __numerales cardinales__, tienen etiqueta de la forma <code>'mc___'</code>. 
- Para números enteros, las etiquetas son <code>'Z'</code>. 
- Las etiquetas que pertenecen a __sustantivos comunes__ son de la forma <code>'nc___'</code>. 
- Las etiquetas que pertenecen a __det. adj.__ son de la forma <code>'da___'</code>. 
- Las etiquetas que pertenecen a __Adj. calificativo__ son de la forma <code>'aq___'</code>.

Entonces, para las cantidades nos quedamos con: 
- Det. indef (ej: un, una): <di.*>
- Det. num (ej: una, dos): <dn.*>
- Num. cardinal (ej: cuatro, cinco): <mc.*>
- Números enteros (ej: 2, 3): <Z\> 

Para las comidas nos quedamos con: 
- Nombre común (ej: tortilla): <nc.\*>

Para los ingredientes nos quedamos con: 
- Nombre común (ej: anchoas): <nc.\*>
- Det. adj (ej: limón): <da.\*>
- Adj. calificativo (ej: margherita): <aq.\*>

In [9]:
grammar = r"""
    cantidad: {<di.*>|<dn.*>|<mc.*>|<Z>}
    comida: {<nc.*>}
    ingrediente: {<nc.*>|<da.*>|<aq.*>}
"""

regex_parser = nltk.RegexpParser(grammar)

In [10]:
#probamos el regex_parser con la gramática que hemos creado

sentence = 'voy a pedir tres bocadillos de anchoas y 2 manzanas'
sentence_tokens = nltk.word_tokenize(sentence)
tagged_sentence = hmm_tagger.tag(sentence_tokens)

r = regex_parser.parse(tagged_sentence)
print(r)

(S
  voy/rg
  a/sps00
  pedir/vmn0000
  (cantidad tres/dn0cp0)
  (comida bocadillos/ncmp000)
  de/sps00
  anchoas/np0000l
  y/cc
  (cantidad 2/Z)
  (comida manzanas/ncmp000))


Vemos que no reconoce anchoas como ingrediente, pero no es problema de la gramática si no del taggeado del hmm por ponerle como etiqueta que es un nombre propio.

Creamos la función que se pide en el enunciado, cuyo input es texto de usuario y devuleve un array con diccionarios de 3 elementos (cantidad, comida e ingredientes).

In [11]:
def regex_IE_comida():
    sentence = input('¿Qué quieres pedir? ')
    
    arr = []
    sentence_tokens = nltk.word_tokenize(sentence)
    tagged_sentence = hmm_tagger.tag(sentence_tokens)
    tree = regex_parser.parse(tagged_sentence)
    
    dic = {'cantidad':1, 'comida':'', 'ingrediente':''}

    for subtree in tree.subtrees():
        
        #hemos terminado de registrar una comida cuando el campo de 'comida' en el diccionario
        #es no vacío y no estamos en un subtree de ingrediente, entonces añadimos el diccionario
        #al array e inicializamos uno nuevo
        if (dic['comida'] != '') & (subtree.label() != 'ingrediente'):
            arr.append(dic)
            dic = {'cantidad':1, 'comida':'', 'ingrediente':''}
            
        if subtree.label() == 'cantidad':
            dic['cantidad'] = subtree[0][0]

        elif subtree.label() == 'comida':
            dic['comida'] = subtree[0][0]

        elif subtree.label() == 'ingrediente':
            dic['ingrediente'] = dic['ingrediente']+', '+subtree[0][0]
            
    arr.append(dic)           
    return arr

In [12]:
#probamos con una frase

regex_IE_comida()

¿Qué quieres pedir? quiero pedir 3 bocadillos de calamares y una ensalada con anchoas


[{'cantidad': '3', 'comida': 'bocadillos', 'ingrediente': ''},
 {'cantidad': 'una', 'comida': 'ensalada', 'ingrediente': ', anchoas'}]

Vemos que no reconoce calamares como ingrediente, vamos a analizar la frase por separado para ver qué puede estar pasando:

In [13]:
sentence = 'quiero pedir 3 bocadillos de calamares y una ensalada con anchoas'

sentence_tokens = nltk.word_tokenize(sentence)
tagged_sentence = hmm_tagger.tag(sentence_tokens)
tree = regex_parser.parse(tagged_sentence)
print(tree)

(S
  quiero/np0000p
  pedir/Fpa
  (cantidad 3/Z)
  (comida bocadillos/ncmp000)
  de/sps00
  calamares/np0000l
  y/cc
  (cantidad una/di0fs0)
  (comida ensalada/ncfs000)
  con/sps00
  (ingrediente anchoas/da0fs0))


El problema está en el tag del hmm ya que ha etiquetado calamares como nombre propio y los ingredientes son nombres comunes, det. adj. o adj. calificativos.

## 4. Se usa el pos tagger y el Regex Parser para obtener las IOB de una nueva frase.

Para generar las IOB de una nueva frase vamos a utilizar la función <code>nltk.chunk.tree2conlltags</code> que genera automáticamente las IOB de un árbol taggeado. 

In [14]:
sentence = 'quiero pedir dos batidos de chocolate y una tarta de limón'
sentence_tokens = nltk.word_tokenize(sentence)
tagged_sentence = hmm_tagger.tag(sentence_tokens)
tree = regex_parser.parse(tagged_sentence)
print(tree2conlltags(tree))

[('quiero', 'sps00', 'O'), ('pedir', 'vmn0000', 'O'), ('dos', 'dn0cp0', 'B-cantidad'), ('batidos', 'ncmp000', 'B-comida'), ('de', 'sps00', 'O'), ('chocolate', 'ncms000', 'B-comida'), ('y', 'cc', 'O'), ('una', 'di0fs0', 'B-cantidad'), ('tarta', 'ncfs000', 'B-comida'), ('de', 'sps00', 'O'), ('limón', 'da0fs0', 'B-ingrediente')]


Vemos que el tagging es correcto ya que los chunks son de __una palabra__ y los taggea como __B-chunk__, si tuvieran más de una palabra la primera sería B-chunk y el resto I-chunk, pero no es el caso. El único error que podríamos detectar es que ha taggeado chocolate como comida y no como ingrediente, pero esto es por la ambigüedad que hay en la descripción de la gramática ya que tanto comidas como ingredientes pueden ser nombres comunes.

Con estos pasos, ya tendríamos cubierta la versión básica de la práctica.

No obstante, por último, se sugiere al alumno que use el pos tagger y el Regex Parser para crear un corpus IOB que sirva para entrenar los bigram taggers o el NaiveBayesClassifier.

Este corpus debe contener las __frases__ con las queremos entrenar y testear __taggeadas con el RegexpParser__ para que encuentre los chunks y en __forma de árbol__, ya que en el entrenamiento el tagger pasara de árbol a tripletas (word, POS, IOB) con la función <code>tree2conlltags</code> y en el evaluate generará una nueva etiqueta IOB con el tagger que ha creado, lo convertirá en un nuevo árbol y lo comparará con el árbol correspondiente del corpus de test.

In [15]:
parsedCorpus = []

for sentence in corpus:

    sentence_tokens = nltk.word_tokenize(sentence)
    tagged_sentence = hmm_tagger.tag(sentence_tokens)
    tree = regex_parser.parse(tagged_sentence)
    #añadimos el árbol directamente ya que los taggers entrenan con árboles con los chunks etiquetados
    parsedCorpus.append(tree)

In [16]:
for tree in parsedCorpus[:3]:
    print(tree)

(S
  Me/pp1cs000
  pones/vmip3s0
  (cantidad un/di0ms0)
  (comida pincho/ncms000)
  de/sps00
  (comida tortilla/ncfs000)
  ,/Fc
  por/sps00
  (comida favor/ncms000)
  ?/Fit)
(S
  quiero/sps00
  (cantidad una/di0fs0)
  (comida pizza/ncfs000)
  (ingrediente margherita/aq0fs0))
(S
  quiero/sps00
  (cantidad un/di0ms0)
  (comida perrito/ncms000)
  (ingrediente caliente/aq0cs0)
  con/sps00
  (comida patatas/ncfp000))


## Train - test

Separamos el corpus en 75% train 25% test

In [17]:
corpus_train = parsedCorpus[:105]
corpus_test = parsedCorpus[105:]

## UnigramTagger

Vamos a entrenar un Unigramtagger con el corpus etiquetado en forma de árbol.

Utilizamos el código disponible en el capítulo 7 del libro de NLTK.

In [18]:
class UnigramChunker(nltk.ChunkParserI):   
    def __init__(self, train_sents):
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        self.tagger = nltk.UnigramTagger(train_data)

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)

In [19]:
unigram_chunker = UnigramChunker(corpus_train)

In [20]:
print(unigram_chunker.evaluate(corpus_test))

ChunkParse score:
    IOB Accuracy: 100.0%%
    Precision:    100.0%%
    Recall:       100.0%%
    F-Measure:    100.0%%


Hemos obtenido el 100% en todas las medidas, es decir, el UnigramTagger ha predicho correctamente todas las POS tags y IOB tags de los tokens del corpus de test (100% en IOB Accuracy). Así como todos los chunks están identificados correctamente también (100% en Precision, Recall y F-measure).

Se espera, por tanto, que tanto el BigramTagger como el NaiveBayesClassifier obtengan estos resultados.

## BigramTagger

Vamos a entrenar un Bigramtagger con el corpus etiquetado en forma de árbol.

Adpatamos el código disponible en el capítulo 7 del libro de NLTK.

In [21]:
class BigramChunker(nltk.ChunkParserI):
    def __init__(self, train_sents):
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        self.tagger = nltk.BigramTagger(train_data)

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)

In [22]:
bigram_chunker = BigramChunker(corpus_train)

In [23]:
print(bigram_chunker.evaluate(corpus_test))

ChunkParse score:
    IOB Accuracy:  94.8%%
    Precision:    100.0%%
    Recall:        89.9%%
    F-Measure:     94.7%%


Vemos que el IOB accuracy es casi del 100% lo cual indica que ha debido de tener algún error al taggear alguna POS tag o IOB tag.

Vemos que Precision es 100% pero Recall no llega al 100% por lo que no debe haber False Positives pero sí False Negatives.

Esto podría corregirse añadiendo un backoff al BigramChunker. Sin embargo, se ha intentado añadir un backoff modificando el código que lo inicializa de la siguiente manera: 

<code>def __init__(self, train_sents, backoff):
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        self.tagger = nltk.BigramTagger(train_data, backoff = backoff)</code>
        
y entrenándolo:

<code>bigram_chunker = BigramChunker(corpus_train, backoff = unigram_chunker)</code>

Sin embargo, esto produce un error de tipo:

<code>AttributeError: 'UnigramChunker' object has no attribute '_taggers'</code>

La función que se pide en el enunciado es análoga a la que se ha creado para el regexpParser:

In [24]:
def bigram_IE_comida():
    sentence = input('¿Qué quieres pedir? ')
    
    arr = []
    sentence_tokens = nltk.word_tokenize(sentence)
    tagged_sentence = hmm_tagger.tag(sentence_tokens)
    tree = bigram_chunker.parse(tagged_sentence)

    dic = {'cantidad':1, 'comida':'', 'ingrediente':''}

    for subtree in tree.subtrees():
        
        #hemos terminado de registrar una comida cuando el campo de 'comida' en el diccionario
        #es no vacío y no estamos en un subtree de ingrediente, entonces añadimos el diccionario
        #al array e inicializamos uno nuevo
        if (dic['comida'] != '') & (subtree.label() != 'ingrediente'):
            arr.append(dic)
            dic = {'cantidad':1, 'comida':'', 'ingrediente':''}
            
        if subtree.label() == 'cantidad':
            dic['cantidad'] = subtree[0][0]

        elif subtree.label() == 'comida':
            dic['comida'] = subtree[0][0]

        elif subtree.label() == 'ingrediente':
            dic['ingrediente'] = dic['ingrediente']+', '+subtree[0][0]
            
    arr.append(dic)           
    return arr

In [25]:
bigram_IE_comida()

¿Qué quieres pedir? quiero pedir 3 bocadillos de calamares y una ensalada con anchoas


[{'cantidad': '3', 'comida': 'bocadillos', 'ingrediente': ''},
 {'cantidad': 'una', 'comida': 'ensalada', 'ingrediente': ', anchoas'}]

Al igual que pasaba con el regexpParser no ha reconocido calamares como ingrediente ya que la etiqueta POS dice que es un nombre propio.

## NaiveBayesClassifier 

Vamos a entrenar un NaiveBayesClassifier con el corpus etiquetado en forma de árbol.

Adaptamos el código disponible en el capítulo 7 del libro de NLTK.

In [26]:
class ConsecutiveNPChunkTagger(nltk.TaggerI):

    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i, (word, tag) in enumerate(tagged_sent):
                featureset = npchunk_features(untagged_sent, i, history)
                train_set.append( (featureset, tag) )
                history.append(tag)
        self.classifier = nltk.NaiveBayesClassifier.train(
            train_set)

    def tag(self, sentence):
        history = []
        for i, word in enumerate(sentence):
            featureset = npchunk_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)

class ConsecutiveNPChunker(nltk.ChunkParserI):
    def __init__(self, train_sents):
        tagged_sents = [[((w,t),c) for (w,t,c) in
                         nltk.chunk.tree2conlltags(sent)]
                        for sent in train_sents]
        self.tagger = ConsecutiveNPChunkTagger(tagged_sents)

    def parse(self, sentence):
        tagged_sents = self.tagger.tag(sentence)
        conlltags = [(w,t,c) for ((w,t),c) in tagged_sents]
        return nltk.chunk.conlltags2tree(conlltags)

Lo único que falta es el extractor de features (etiqueta POS):

In [27]:
def npchunk_features(sentence, i, history):
    word, pos = sentence[i]
    return {"pos": pos}

In [28]:
chunker = ConsecutiveNPChunker(corpus_train)

In [29]:
print(chunker.evaluate(corpus_test))

ChunkParse score:
    IOB Accuracy: 100.0%%
    Precision:    100.0%%
    Recall:       100.0%%
    F-Measure:    100.0%%


Como se esperaba todas las medidas han resultado en el 100%.

La función que se pide en el enunciado es análoga a la que se ha creado para el regexpParser:

In [30]:
def naive_IE_comida():
    sentence = input('¿Qué quieres pedir? ')
    
    arr = []
    sentence_tokens = nltk.word_tokenize(sentence)
    tagged_sentence = hmm_tagger.tag(sentence_tokens)
    tree = chunker.parse(tagged_sentence)

    dic = {'cantidad':1, 'comida':'', 'ingrediente':''}

    for subtree in tree.subtrees():
        
        #hemos terminado de registrar una comida cuando el campo de 'comida' en el diccionario
        #es no vacío y no estamos en un subtree de ingrediente, entonces añadimos el diccionario
        #al array e inicializamos uno nuevo
        if (dic['comida'] != '') & (subtree.label() != 'ingrediente'):
            arr.append(dic)
            dic = {'cantidad':1, 'comida':'', 'ingrediente':''}
            
        if subtree.label() == 'cantidad':
            dic['cantidad'] = subtree[0][0]

        elif subtree.label() == 'comida':
            dic['comida'] = subtree[0][0]

        elif subtree.label() == 'ingrediente':
            dic['ingrediente'] = dic['ingrediente']+', '+subtree[0][0]
            
    arr.append(dic)           
    return arr

In [31]:
naive_IE_comida()

¿Qué quieres pedir? quiero pedir 3 bocadillos de calamares y una ensalada con anchoas


[{'cantidad': '3', 'comida': 'bocadillos', 'ingrediente': ''},
 {'cantidad': 'una', 'comida': 'ensalada', 'ingrediente': ', anchoas'}]

Al igual que pasaba con el regexpParser no ha reconocido calamares como ingrediente ya que la etiqueta POS dice que es un nombre propio.